In [ ]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import librosa
import os
from audio_dataset import audio_segment
os.chdir(r"C:\Users\owner\Documents\Repos\Text2BGAudio")

### Music Dataset

In [8]:
music_data_raw_folder = r"_Data/Music/raw"
save_folder = r"_Data/Music"
SR = 8000

In [12]:
data_set = []
for label in os.listdir(music_data_raw_folder):
    curr_size = len(data_set)
    title = f"Processing label: {label}"
    print("#" * len(title))
    print(title)
    label_folder_path = os.path.join(music_data_raw_folder, label)
    for audio_file in os.listdir(label_folder_path):
        audio_file_path = os.path.join(label_folder_path, audio_file)
        waveform, orig_sample_rate = librosa.load(audio_file_path)
        waveform = torch.tensor(waveform)
        if orig_sample_rate != SR:
            if orig_sample_rate < SR:
                print(
                    f"Sample rate of {orig_sample_rate} is less than target sample rate {SR}. Skipping {audio_file_path}"
                )
                raise
            resampler = torchaudio.transforms.Resample(
                orig_freq=orig_sample_rate, new_freq=SR
            )
            waveform = resampler(waveform)
        segment_length = 10 * SR
        batches = waveform.split(segment_length)
        for batch_seg in batches:
            if len(batch_seg) < segment_length:
                continue
            data_set.append(audio_segment(data=batch_seg, label=label, org_file=audio_file_path))
    print(f"Created {len(data_set) - curr_size} segments")
    print("#" * len(title)  + "\n\n")

torch.save(data_set, os.path.join(save_folder, f"music_dataset_size{len(data_set)}_SR_{SR}.pt"))

#######################
Processing label: Angry
Created 120 segments
#######################


#####################
Processing label: Joy
Created 215 segments
#####################


#####################
Processing label: Sad
Created 154 segments
#####################


#######################
Processing label: Scary
Created 161 segments
#######################


